## Building a Database AI Agent with Streamlit, Python, and NexusRavenV2-13B LLM

In the world of data-driven decision-making, the ability to query and interact with databases efficiently can significantly streamline workflows. In this project, I developed a Database AI Assistant that allows users to interact with databases using natural language, making database queries more accessible, especially for non-technical users. The app leverages Streamlit for the frontend, Python for the backend, and NexusRavenV2-13B, a cutting-edge Large Language Model (LLM), to handle natural language processing (NLP) and SQL generation. By utilizing function calling and tools, I built a web app solution that transforms conversational input into SQL queries, all while maintaining a high degree of accuracy and performance.

In this post, I'll walk you through the process, and the reasoning behind my design choices. If you're a developer, data analyst, or someone looking to simplify database interactions, this might be the solution you need.
Below is the snapshot of the Database AI Chatbot:


![SNOWFALL](appimage.png)


### Why NexusRavenV2-13B?

I chose NexusRavenV2-13B because of its powerful natural language understanding and ability to generate high-quality responses based on context. NexusRavenV2-13B excels at understanding complex queries and interpreting user intent, making it ideal for translating user input into SQL queries.

What sets this version apart is its ability to handle function calling. Instead of generating SQL from scratch every time, which comes with its own cost and latency, the agent leverages predefined SQL functions that are optimized for common tasks. This ensures that queries are more efficient, especially when dealing with large databases or complex operations.

### Why Streamlit?

For the frontend, I used Streamlit, which is a fast, open-source framework for building web apps with minimal effort. Streamlit is perfect for projects like this, where you want to focus on functionality without spending much time on complex UI development. The UI is clean, interactive, and easy to extend, which made it ideal for this project.

With Streamlit, I was able to create:

1. A user-friendly interface to input natural language queries.
2. Real-time updates for displaying query results and insights.
3. Minimal overhead by handling most of the logic in Python, keeping the codebase concise.

### Key Features of the Database AI Agent

##### Natural Language Processing (NLP): 
The agent can understand a wide range of questions and queries in plain English, from simple requests like "Show me the sales data from last month" to more complex queries involving joins or aggregate functions.

##### Function Calling for SQL Optimization: 
By using function calls and predefined tools, the agent ensures that the generated SQL queries are not only syntactically correct but also optimized for the task at hand.

##### Real-time Query Execution: 
Once the SQL query is called based on the user input, the agent then generates the SQL query using the predefined function and executes it on the connected database and the results are displayed immediately with low latency. 


#### Here's a simplified example of the core Python code that drives the logic:

In [ ]:
import streamlit as st
import sqlite3
import random

# Streamlit app configuration
st.set_page_config(page_title="DatabaseChatbot", page_icon="🤖", layout="centered")

# Title of the app
st.title("Database AI Chatbot")


# Creating the functions
def query(payload):
    """
    Sends a payload to a TGI endpoint.
    """
    API_URL = "http://nexusraven.nexusflow.ai"
    headers = {
        "Content-Type": "application/json"
    }
    import requests
    response = requests.post(API_URL, headers=headers, json=payload)
    return response.json()

def query_raven(prompt):
    """
    This function sends a request to the TGI endpoint to get Raven's function call.
    This will not generate Raven's justification and reasoning for the call, to save on latency.
    """
    import requests
    output = query({
        "inputs": prompt,
        "parameters" : {"temperature" : 0.001, "stop" : ["<bot_end>"], "do_sample" : False, "max_new_tokens" : 2048, "return_full_text" : False}})
    call = output[0]["generated_text"].replace("Call:", "").strip()
    return call

def execute_sql(sql_code : str):
    import sqlite3
    
    # Connect to the database
    conn = sqlite3.connect('toy_database.db')
    cursor = conn.cursor()
    
    cursor.execute('PRAGMA table_info(toys)')
    columns = [info[1] for info in cursor.fetchall()]  # Extracting the column names
    
    # Query to select all data
    cursor.execute(sql_code)
    rows = cursor.fetchall()
    
    return_string = " ".join(columns)
    for idx, row in enumerate(rows):
        row = (idx, *row)
        return_string += "\n" + str(row)
    
    # Close the connection
    conn.close()
    return return_string


# Internal database name setting
# create_random_database()
DB_NAME = 'toy_database.db'

# Connect to the database
def connect_db():
    return sqlite3.connect(DB_NAME)

# List all toys
def list_all_toys():
    with connect_db() as conn:
        cursor = conn.execute('SELECT * FROM toys')
        return cursor.fetchall()

# Function to get the most expensive toy
def get_most_expensive_toy(count=1):
    with connect_db() as conn:
        cursor = conn.execute(f'SELECT * FROM toys ORDER BY price DESC LIMIT {count}')
        return cursor.fetchone()

# Function to get the cheapest toy
def get_cheapest_toy(count=1):
    with connect_db() as conn:
        cursor = conn.execute(f'SELECT * FROM toys ORDER BY price ASC LIMIT {count}')
        return cursor.fetchone()


def get_raven_response(question):
    try:
        raven_prompt = \
            f'''
            Function:
            def list_all_toys():
                """
                Retrieves a list of all toys from the database. This function does not take any parameters.
                Returns: A list of tuples, where each tuple represents a toy with all its attributes (id, name, price).
                """

            Function:
            def get_most_expensive_toy(count : int):
                """
                Retrieves the most expensive toy from the database.
                This function does not take any parameters.

                Returns: A tuple representing the most expensive toy, including its id, name, and price.
                """

            Function:
            def get_cheapest_toy(count : int):
                """
                Finds and retrieves the cheapest toy in the database.
                This function does not take any parameters.

                Returns: A tuple representing the cheapest toy, including its id, name, and price.
                """

            User Query: {question}<human_end>

            '''


        output = query_raven(raven_prompt)
        # print (f"LLM's function call: {output}")
        database_result = eval(output)

        full_prompt = \
        f"""
        <s> [INST]
        {database_result}

        Use the information above to answer the following question in a single sentence.

        Question:
        {question} [/INST]
        """
        # Make a request to the raven api for generating a response
        grounded_response = query_raven(full_prompt)
        return grounded_response
    except Exception as e:
        return f"Error: {str(e)}"


# Initialize the session state for the conversation
if "messages" not in st.session_state:
    st.session_state.messages = []

# Display chat messages from history on app rerun
for message in st.session_state.messages:
    with st.chat_message(message["role"]):
        st.markdown(message["content"])

# Accept user input
if prompt := st.chat_input("Message Database AI Assistant!"):
    # Add user message to chat history
    st.session_state.messages.append({"role": "user", "content": prompt})
    # Display user message in chat message container
    with st.chat_message("user"):
        st.markdown(prompt)

    # Display assistant response in chat message container
    with st.chat_message("assistant"):
        question = st.session_state.messages[-1]["content"]

        answer = get_raven_response(question)
        # translated_answer = .... 
        response = st.write(answer)
    st.session_state.messages.append({"role": "assistant", "content": answer})  


### Future Improvements

##### While the app is already functional, there are several areas for future enhancement:

1. Advanced Query Support: Adding more complex SQL functionality, like subqueries or window functions, to handle more sophisticated use cases.
2. User Personalization: Incorporating user-specific preferences or history to tailor SQL generation to different workflows or data models.
3. Security and Access Control: Implementing user authentication to ensure secure database access, especially when dealing with sensitive data.
4. Query Optimization Insights: Providing feedback on how to optimize queries for better performance or suggesting database indexing strategies.
5. Adding necessary guardrails to catch any errant behavior by the model.

### Limitations

1. A large number of functions will saturate the context window of NexusRaven-V2 .
2. The model can be prone to generate incorrect calls hence there is a need for proper guardrails to capture errant behavior is in place.

### Conclusion

The Database AI Agent App is a powerful tool that simplifies database interactions by allowing users to communicate with their databases in natural language. By combining Streamlit’s ease of use, NexusRavenV2-13B’s advanced NLP capabilities, and function calling for optimized SQL generation, this project demonstrates the potential of AI to enhance productivity and streamline complex tasks.

You can find the source code on GitHub and feel free to connect with me on LinkedIn to discuss improvements, feedback, or potential collaborations.



#### Feedback & Contributions: 
I welcome any suggestions, bug reports, or contributions to enhance this project. \
    GitHub: https://github.com/ola-sam \
    LinkedIn: https://www.linkedin.com/in/samelegure/